Import the dependencies

In [2]:
import numpy as np
import pandas as pd
import re  # This library helps find and work with specific patterns in text, like finding all words that start with "a".
from nltk.corpus import stopwords  # This gives us a list of common words like 'in', 'the', and 'to' that usually don't add much meaning to sentences. We can remove these to focus on the important words.
from nltk.stem.porter import PorterStemmer  # This tool helps us reduce words to their basic form. For example, it changes 'running' to 'run', so we can treat them as the same word.
from sklearn.feature_extraction.text import TfidfVectorizer  # This tool turns text into numbers that represent how important each word is in a bunch of documents. This helps computers understand and compare the text more easily.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
# printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [10]:
# loading the dataset to the pandas dataframe
news_dataset = pd.read_csv('/content/train.csv')


In [11]:
news_dataset.shape

(20800, 5)

In [12]:
# print 5 rows of dataframe
print(news_dataset.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [13]:
# Counting the number of missing values in dataframe
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [14]:
# Replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [17]:
# Merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [18]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [19]:
# Seperating the data and labels
x = news_dataset.drop(columns = 'label', axis = 1)
y = news_dataset['label']


In [20]:
print(x)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [21]:
print(y)

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64


Stemming:
Stemming is the process of reducing a word to its root word

Example:
actor, actress, acting ---> act

In [22]:
port_stem = PorterStemmer()

In [25]:
def stemming(content):
  # This function takes a piece of text as input and returns a simplified version of it.

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  # Replace anything that's not a letter (a-z or A-Z) with a space. This helps to remove numbers and special characters.

  stemeed_content = stemmed_content.lower()
  # Convert all letters in the text to lowercase, so 'Word' and 'word' are treated the same.

  stemmed_content = stemmed_content.split()
  # Split the text into a list of words. For example, 'Hello world' becomes ['hello', 'world'].

  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # For each word in the list, if the word is not a common word like 'the' or 'in' (stopwords), reduce it to its root form.
  # For example, 'running' becomes 'run'.

  stemmed_content = ' '.join(stemmed_content)
  # Join the list of words back into a single string, with words separated by spaces.

  return stemmed_content
  # Return the processed text.


In [26]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [27]:
print(news_dataset['content'])

0        darrel lucu hous dem aid we didn even see come...
1        daniel j flynn flynn hillari clinton big woman...
2           consortiumnew com whi truth might get you fire
3        jessica purkiss civilian kill in singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t i trump poster child for...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci is said...
20798    alex ansari nato russia to hold parallel exerc...
20799                       david swanson what keep f aliv
Name: content, Length: 20800, dtype: object


In [28]:
# separating the data and label
x = news_dataset['content'].values
y = news_dataset['label'].values

In [29]:
print(x)

['darrel lucu hous dem aid we didn even see comey letter until jason chaffetz tweet it'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get you fire' ...
 'michael j de la merc rachel abram maci is said receiv takeov approach hudson bay the new york time'
 'alex ansari nato russia to hold parallel exercis in balkan'
 'david swanson what keep f aliv']


In [30]:
print(y)

[1 0 1 ... 0 1 1]


In [31]:
y.shape

(20800,)

In [32]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(x)

x = vectorizer.transform(x)

In [33]:
print(x)

  (0, 16586)	0.19235766814206556
  (0, 16055)	0.29053629972079575
  (0, 15762)	0.25180682754084327
  (0, 13528)	0.22682424464275375
  (0, 8949)	0.3214177525383042
  (0, 8670)	0.25823746120670493
  (0, 7731)	0.219100353547038
  (0, 7639)	0.1611238443347535
  (0, 7040)	0.19336678366028098
  (0, 4998)	0.20625126961544935
  (0, 4028)	0.2667908047240904
  (0, 3811)	0.23915031396884368
  (0, 3619)	0.31814479104571186
  (0, 2977)	0.21821000835755877
  (0, 2501)	0.3250028762214839
  (0, 272)	0.2387684291853839
  (1, 16892)	0.30071745655510157
  (1, 6849)	0.1904660198296849
  (1, 5528)	0.7143299355715573
  (1, 3587)	0.26373768806048464
  (1, 2831)	0.19094574062359204
  (1, 2241)	0.3827320386859759
  (1, 1910)	0.15521974226349364
  (1, 1512)	0.2939891562094648
  (2, 17095)	0.29337976465513754
  :	:
  (20797, 9631)	0.17242189281191916
  (20797, 9561)	0.2918128273796141
  (20797, 9030)	0.35719284755530417
  (20797, 8404)	0.22049990081059304
  (20797, 7611)	0.13993333005918895
  (20797, 7077)	0.215

Splitting the dataset to training & test data

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 2)
# stratify=y: This ensures that the proportion of different classes in the target variable y is the same in both the training and testing sets.
# For example, if 30% of the data belongs to class 1 and 70% to class 0, this ratio will be maintained in both x_train and x_test.

In [37]:
print(x.shape, x_train.shape, x_test.shape)

(20800, 17227) (16640, 17227) (4160, 17227)


Training the Model - Logistic Regression

In [39]:
model = LogisticRegression()

In [40]:
model.fit(x_train, y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [41]:
# Accuracy Score on training data
x_train_pred = model.predict(x_train)
y_train_accuracy = accuracy_score(x_train_pred, y_train)

In [42]:
print('Accuracy score of training data : ', y_train_accuracy)

Accuracy score of training data :  0.9868990384615385


In [43]:
# Accuracy Score on test data
x_test_pred = model.predict(x_test)
y_test_accuracy = accuracy_score(x_test_pred, y_test)

In [44]:
print('Accuracy score of test data : ', y_test_accuracy)

Accuracy score of test data :  0.9762019230769231


Making a Predictive System

In [50]:
x_new = x_test[1]

prediction = model.predict(x_new)
print(prediction)

if prediction[0] == 0:
  print('The news is real!')
else:
  print('The news is fake!')

[0]
The news is real!


In [49]:
print(y_test[1])

0
